In [1]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np
import shapely

In [2]:
PROJECTION=32630

#### Output area geometry

In [3]:
oa_geom = gpd.read_file('../data/london/shapes/oa/OA_2011_London_gen_MHW.shp')
oa_geom.to_crs(epsg=PROJECTION, inplace=True)
oa_geom.columns = oa_geom.columns.str.lower()
# Some invalid geoms. Fix them.
oa_geom.geometry = oa_geom.buffer(0)

#### Output area census

In [4]:
ages = [str(age) for age in range(18,65)]
oa_pop = pd.read_csv('../data/london/census/output_area_mid-2017-estimates-london.csv', skiprows=4, usecols=['OA11CD', 'LSOA11CD', 'All Ages']+ages)
oa_pop.rename(columns={'All Ages':'population'}, inplace=True)
oa_pop.columns = oa_pop.columns.str.lower()

# Remove apostrophes for thousand separators
oa_pop.population = oa_pop.population.str.replace("'","")
oa_pop.population = oa_pop.population.astype(int)

# Create adult population
oa_pop['adult_population'] = oa_pop[ages].sum(axis=1)
oa_pop = oa_pop.drop(columns=ages)

#### Station geometry

In [5]:
station_geom = gpd.read_file('../data/london/shapes/stations/stations_mod.shp')
# Change to UTM
station_geom.to_crs(epsg=PROJECTION, inplace=True)
# Remove the word "Station"
station_geom.columns = station_geom.columns.str.lower()
station_geom.rename(columns={'geometry':'location'}, inplace=True)
station_geom.name = station_geom.name.str.replace('\t','').str.replace('\n','').str.replace(' Station','')
station_geom.set_index('name', drop=True, inplace=True)

# Read Voronoi cells
station_voronoi = gpd.read_file('../data/london/shapes/stations/voronoi.shp')
station_voronoi.to_crs(epsg=PROJECTION, inplace=True)
station_voronoi.columns = station_voronoi.columns.str.lower()
station_voronoi.rename(columns={'geometry':'cell'}, inplace=True)
station_voronoi.name = station_voronoi.name.str.replace('\t','').str.replace('\n','').str.replace(' Station','')
station_voronoi.set_index('name', drop=True, inplace=True)

# Combine Voronoi with location
station_geom['cell'] = station_voronoi.reindex(station_geom.index).cell

#### Station ridership

In [6]:
counts_en = pd.read_csv('/Users/itto/Documents/cities/data/london/counts/En17week_mod.csv', skiprows=6).set_index(' Station')
counts_ex = pd.read_csv('/Users/itto/Documents/cities/data/london/counts/Ex17week.csv', skiprows=6).set_index(' Station')

# We only care about the time columns
time_columns = counts_en.columns[4:-8]

# Easier to work with stations as columns, so transpose
data_en = counts_en.loc[:, time_columns].T
data_ex = counts_ex.loc[:, time_columns].T

# Remove "Total" column
data_en.drop('Total',axis=1,inplace=True)
data_ex.drop('Total',axis=1,inplace=True)

# Extract the first time
data_en.index = data_en.index.str[:4]
data_ex.index = data_en.index.str[:4]

# Convert to datetime format
data_en.index = pd.to_datetime(data_en.index, format='%H%M')
data_ex.index = pd.to_datetime(data_en.index, format='%H%M')

# Create a time of day index that we can use from now on
time_of_day = {
    6: 'morning',
    7: 'morning',
    8: 'morning',
    9: 'morning',
    11: 'off',
    12: 'off',
    13: 'off',
    14: 'off',
    16: 'afternoon',
    17: 'afternoon',
    18: 'afternoon',
    19: 'afternoon'
}
time_of_day = data_en.index.hour.map(time_of_day).to_series()
time_of_day[time_of_day.isnull()] = 'night'
time_of_day = time_of_day.values

In [7]:
# Get the entrances and exists for each time interval
entrances = data_en.groupby(time_of_day).sum().T
exits = data_ex.groupby(time_of_day).sum().T

#### Combine the data

In [8]:
# Combine population and ward geometry
output_areas = pd.merge(oa_geom, oa_pop[['oa11cd','population','adult_population']], on=['oa11cd'])
output_areas['density_actual'] = output_areas.population/output_areas.area * 1000000

In [9]:
%%time
wards = output_areas.dissolve('wd11cd_bf', aggfunc='sum', as_index=False)
wards['density_actual'] = wards.population/wards.area * 1000000

In [10]:
%%time
boroughs = output_areas.dissolve('lad11nm', aggfunc='sum', as_index=False)
boroughs['density_actual'] = boroughs.population/boroughs.area * 1000000

In [13]:
# Combine entrances and station locations data
stations = pd.merge(station_geom, entrances, left_index=True, right_index=True)
stations['station'] = stations.index

#### Preprocess station zones

In [14]:
# Buffer distance
stations['zone'] = stations.set_geometry('location').buffer(500)

In [15]:
# Extract some helpful geometries
shp_london = gpd.read_file('../data/london/shapes/misc/london.shp')
box = shapely.geometry.box(*boroughs.geometry.total_bounds.tolist()).buffer(2000)
box = gpd.GeoDataFrame({'geometry':[box]})
box.crs = stations.crs
external = gpd.overlay(box, boroughs, how='difference')
water = gpd.overlay(external, shp_london, how='intersection')
external = gpd.overlay(external, water, how='difference')
london_nowater = gpd.GeoDataFrame({'geometry':boroughs.union(external)},crs=stations.crs)

/Users/itto/miniconda3/envs/spatial/lib/python3.6/site-packages/geopandas/base.py:29: UserWarning: GeoSeries crs mismatch: {'init': 'epsg:32630', 'no_defs': True} and {'init': 'epsg:32630'}
  other.crs))


In [16]:
# Voronoi cells encompass a large area. Restrict them to London.
stations = gpd.overlay(stations.set_geometry('cell'), box)
stations.rename(columns={'geometry':'cell'},inplace=True)

In [17]:
# Restrict station zones to land
stations = gpd.overlay(stations.set_geometry('zone'), water, how='difference')
stations = stations.dissolve('station', as_index=False)
stations.rename(columns={'geometry':'zone'},inplace=True)

In [18]:
# Restrict zones to be within Voronoi cell
n = stations.shape[0]
stations = gpd.overlay(stations.set_geometry('zone'), stations.loc[:,['station','cell']].set_geometry('cell'))
stations.rename(columns={'geometry':'zone'},inplace=True)
# Each zone overlaps with multiple cells. Only get the cell corresponding to the zone's station.
stations = stations.loc[stations.station_1 == stations.station_2]
# We have duplicate columns. Keep only one. 
stations.drop('station_2',axis=1,inplace=True)
stations.rename(columns={'station_1':'station'},inplace=True)
stations.set_index('station',drop=True,inplace=True)
print('Lost {} rows while overlapping geometries'.format(n - stations.shape[0]))

Lost 1 rows while overlapping geometries
